In [1]:
import numpy as np
import random
import torch
from src.dataloaders import Dataset100k
from src.models import MLPBCEModel
from src.trainer import Trainer
from src.metrics import hitratio, ndcg

np.random.seed(42)
random.seed(42)
torch.manual_seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = 'cpu'
print(f"{device=}")

device='cuda'


In [2]:
class config:
    data_dir = "ml-100k"
    epochs = 12
    batch_size = 2048
    dim = 32
    lr = 0.005
    b1 = 0.3
    b2 = 0.6
    weight_decay = 0.0001


dataset = Dataset100k(config.data_dir)
dataset.gen_adjacency()
dataset.make_train_test()
print(f"{dataset.train_size=}, {dataset.test_size=}")

metrics = {
    "HR@1": (hitratio, {"top_n": 1}),
    "HR@5": (hitratio, {"top_n": 5}),
    "HR@10": (hitratio, {"top_n": 10}),
    "NDCG@1": (ndcg, {"top_n": 1}),
    "NDCG@5": (ndcg, {"top_n": 5}),
    "NDCG@10": (ndcg, {"top_n": 10}),
}

dataset.train_size=198114, dataset.test_size=943


In [3]:
model = MLPBCEModel(
    dataset.user_count, dataset.item_count, embed_size=config.dim, layers=[32, 16, 8]
)

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=config.lr,
    betas=(config.b1, config.b2),
    weight_decay=config.weight_decay,
)

trainer = Trainer(
    dataset,
    model,
    optimizer,
    metrics,
    epochs=config.epochs,
    batch_size=config.batch_size,
    device=device,
)

In [4]:
trainer.train(evaluate=True, verbose=True, progressbar=True)
# trainer.test(verbose=False, pbar=False)

Epoch 0: Avg Loss/Batch 0.646106            


HR@1: 0.07953340402969247
HR@5: 0.3064687168610817
HR@10: 0.4676564156945917
NDCG@1: 0.07953340402969247
NDCG@5: 0.19524336098642198
NDCG@10: 0.2477315970657404


Epoch 1: Avg Loss/Batch 0.513093            


HR@1: 0.11876988335100742
HR@5: 0.3647932131495228
HR@10: 0.5397667020148462
NDCG@1: 0.11876988335100742
NDCG@5: 0.24503404486294067
NDCG@10: 0.30098859787891613


Epoch 2: Avg Loss/Batch 0.485928            


...

HR@1: 0.24284199363732767
HR@5: 0.5715800636267232
HR@10: 0.7306468716861082
NDCG@1: 0.24284199363732767
NDCG@5: 0.41369451010487895
NDCG@10: 0.4649406565223062


Epoch 10: Avg Loss/Batch 0.382559            


HR@1: 0.2523860021208908
HR@5: 0.5758218451749735
HR@10: 0.7295864262990456
NDCG@1: 0.2523860021208908
NDCG@5: 0.4212031301742806
NDCG@10: 0.4710435018678831


Epoch 11: Avg Loss/Batch 0.377677            


HR@1: 0.2492046659597031
HR@5: 0.574761399787911
HR@10: 0.7401908801696713
NDCG@1: 0.2492046659597031
NDCG@5: 0.42132259848599796
NDCG@10: 0.47524326200356


In [5]:
best_epoch = np.argmax([r["NDCG@10"] for r in trainer.test_log])
print(f"{best_epoch}: {trainer.test_log[best_epoch]}")

11: {'HR@1': 0.2492046659597031, 'HR@5': 0.574761399787911, 'HR@10': 0.7401908801696713, 'NDCG@1': 0.2492046659597031, 'NDCG@5': 0.42132259848599796, 'NDCG@10': 0.47524326200356}


In [6]:
torch.save(trainer.model.state_dict(), "saved_models/mlpbce.pt")
# trainer.model.load_state_dict(torch.load("saved_models/mlpbce.pt"))